In [66]:
import bs4
import asyncio
import aiohttp
import pandas as pd
import time
import nest_asyncio

In [4]:
df = pd.read_excel('./Fin_Res_Q3_22_23_12_Feb_2023_in_V1.xlsx')
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
urls = df["res_url"].tolist()

In [5]:
outdf = pd.DataFrame(columns=["SYMB", " DEC '22_R", " DEC '22_V", " DEC '22_DEPS",
                              " DEC '22_Interest", " DEC '22_Gross NPA", " DEC '22_Net NPA", "LUU",
                              "RES_CATG_1", "RES_CATG_2", "L_RES_DT", "DATA_AVAILABLE",
                              "Last Available Data", "TIME STAMP", "RES_URL", "SECTOR"])

def fill_df(url,data):
        if len(data) == 0:
            outdf.loc[urls.index(url), "RES_URL"] = df.loc[urls.index(url)+1, "res_url"]
            outdf.loc[urls.index(url), "DATA_AVAILABLE"] = "N"
            outdf.loc[urls.index(url), "TIME STAMP"] = time.strftime("%d/%m/%Y %H:%M:%S")
            outdf.loc[urls.index(url), "SECTOR"] = df.loc[urls.index(url)+1, "sectr"]
            outdf.loc[urls.index(url),'SYMB'] = df.loc[urls.index(url)+1, "symb"]
            outdf.loc[urls.index(url), "LUU"] = df.loc[urls.index(url)+1, "LUU"]
            outdf.loc[urls.index(url), "RES_CATG_1"] = df.loc[urls.index(url)+1, "res_catg1"]
            outdf.loc[urls.index(url), "RES_CATG_2"] = df.loc[urls.index(url)+1, "res_catg2"]
            outdf.loc[urls.index(url), "L_RES_DT"] = df.loc[urls.index(url)+1, "l_res_dt"]
            outdf.loc[urls.index(url), "Last Available Data"] = "NONE"
        elif len(data) > 0:
            dg = pd.DataFrame(data,index=None,columns=None)
            dg.columns = dg.iloc[0]
            dg = dg.drop(dg.index[0])
            dg = dg.iloc[:, :2]
            outdf.loc[urls.index(url),'SYMB'] = df.loc[urls.index(url)+1, "symb"]
            outdf.loc[urls.index(url), "LUU"] = df.loc[urls.index(url)+1, "LUU"]
            outdf.loc[urls.index(url), "RES_CATG_1"] = df.loc[urls.index(url)+1, "res_catg1"]
            outdf.loc[urls.index(url), "RES_CATG_2"] = df.loc[urls.index(url)+1, "res_catg2"]
            outdf.loc[urls.index(url), "L_RES_DT"] = df.loc[urls.index(url)+1, "l_res_dt"]
            outdf.loc[urls.index(url), "RES_URL"] = df.loc[urls.index(url)+1, "res_url"]
            outdf.loc[urls.index(url), "SECTOR"] = df.loc[urls.index(url)+1, "sectr"]
            outdf.loc[urls.index(url), " DEC '22_R"] = dg.loc[1][1]
            outdf.loc[urls.index(url), " DEC '22_V"] = dg.loc[28][1]
            outdf.loc[urls.index(url), " DEC '22_DEPS"] = dg.loc[37][1]
            outdf.loc[urls.index(url), " DEC '22_Interest"] = dg.loc[20][1]
            outdf.loc[urls.index(url), "DATA_AVAILABLE"] = "Y"
            outdf.loc[urls.index(url), "Last Available Data"] = dg.columns[1]
            outdf.loc[urls.index(url), "TIME STAMP"] = time.strftime("%d/%m/%Y %H:%M:%S")
            if outdf.loc[urls.index(url), "SECTOR"] == "Banks":
                outdf.loc[urls.index(url), " DEC '22_R"] = dg.loc[2][1]
                outdf.loc[urls.index(url), " DEC '22_V"] = dg.loc[20][1]
                outdf.loc[urls.index(url), " DEC '22_DEPS"] = dg.loc[30][1]
                outdf.loc[urls.index(url), " DEC '22_Interest"] = "0"
                outdf.loc[urls.index(url), " DEC '22_Gross NPA"] = dg.loc[35][1]
                outdf.loc[urls.index(url), " DEC '22_Net NPA"] = dg.loc[36][1]

In [87]:
html = []

In [101]:
async def get_data():
    async with aiohttp.ClientSession() as session:
        for url in urls[:3]:
            async with session.get(url) as response:
                html.append(await response.text())
            

In [99]:
tasks = asyncio.gather(get_data())
nest_asyncio.apply()
loop = asyncio.get_event_loop()
loop.run_until_complete(tasks)

for i in range(len(html)):
    soup = bs4.BeautifulSoup(html[i], 'html.parser')
    data = []
    if soup.find('table') is None:
        fill_df(urls[i],data)
        continue
    for tr in soup.find_all('tr'):
        tds = tr.find_all('td')
        data.append([td.text.strip() for td in tds])
    fill_df(urls[i],data)

In [90]:
from concurrent.futures import ThreadPoolExecutor
try:
    asyncio.get_running_loop()
    with ThreadPoolExecutor(1) as pool:
        result = pool.submit(lambda: asyncio.run(get_data())).result()
except RuntimeError:
    result = asyncio.run(get_data())